<a href="https://colab.research.google.com/github/FrederiKob/Kelly_Replication_Code/blob/main/Library_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

**The Function Script includes the following functions:**


1.   Generate w_i --> generates random N(0,1) draws used in the RFFs (*generate_w_i*)
2.   Generate Signals --> generates RFF's for a specific seed and for a specific number of P's (*generate_Signals*)
3.   Generate X and y samples used for fitting and predicting (*generate_X_y*)
4.   Run predictions over all t's and over multiple iterations (*run_all*)

1. Generate w_i:
Takes as input:
*   max_iterations: number of iterations to average predictions over
*   max_P: maximum nuber of P's used to generate RFF's (only need half since we are using cos and sin)
*   will always be run using seeds from range(0, max_iterations) --> replicability guaranteed

From Kelly et al. (2022) *The Virtue Complexity Everywhere* page 21:
"[...] given the signal vector $X_{t} = \mathbb{R}^{d}$ [...] fix a random seed $s$ and sample random weights $w_{i}(s) \in \mathbb{R}^{d}$ from $\mathcal{N} \sim (0,I_{dxd})$ "

Hence, the w_i gets drawn from multivariate_normal mean = 0; covariance matrix = Identity.
Resulting in a dictionary with max_iteration number of keys (name  with every key containing a matrix of shape max_P/2 x number of predictors (G = 15).

=> every matrix is generated by the specific seed of range(max_iterations), thus the random draws and predicitons are easily replicated and can be used for all P-T-alpha generations.


In [ ]:
def generate_w_i(max_iterations = None, max_P = None):
    dic_wi = dict.fromkeys(range(max_iterations))
    """ Always set seed to 0 --> generate max_iteration verisons of random draws used
        --> one set of these random w_i's is used for one iteration
    """
    for ite in range(max_iterations):
        np.random.seed(ite)
        # round so that sufficient w_i are drawn in case max_P is odd
        w_i = np.random.multivariate_normal(mean = np.array([0]*15), cov = np.identity(15), size = int(round(max_P/2)))
        dic_wi[ite] = w_i
    return dic_wi

2. generate_signals:
*   pred_std: standardized predictor variables used to generate signals (lagged market return is included)
*   P: number of predictors to generate $P \in [2,...,12000]$

The signals or RFFs are generated as follows (see Kelly et al. (2022)):
 $S_{i,t}(s) = \frac{1}{P^{1/2}}\left [ sin(\gamma w_{i}(s)'X_{t}), cos(\gamma w_{i}(s)' X_{t}) \right ]', i = 1,\cdots, P$,

 where "[...] the parameter $\gamma$ controls the Gaussian kernel bandwitdth in the generation of random Fourier features [...] we set $\gamma = 2$. Our results are generally insensitive to $\gamma$..." (page 39-40)




In [ ]:
def generate_Signals(pred_std = None, P = None, use_seed = None, w_i_all = None):
    # gamma = 2
    gamma = 2
    # use seed picks out dicitonary entries of w_i generated with different seeds in order to use a specific seed = use_seed
    w_i = w_i_all[use_seed][:int(round(P/2))]
    inside = gamma * w_i @ pred_std.T
    # using trig-function
    sig = (P**(-0.5) * np.cos(inside.T), P**(-0.5) * np.sin(inside.T))
    signals = np.concatenate([sig[0], sig[1]], axis=1)
    # check if even or odd
    if P % 2 != 0:
        # random item to drop --> due to one signal too many
        drop = np.random.randint(0, P+1)
        # drop random element --> correct size of signals
        signals = np.matrix([np.delete(i, drop, 0) for i in signals])
    else:
        signals = np.matrix(signals)

    # return signals
    return signals


3. Generate X_y:



*   In line with Kelly et al. page 40: "Prior to estimation, we volatility standardize the training sample RFFs ${S_{t-1}, ..., S_{t-T}}$ and out-of-sample RFFs $S_{t}$ by their standard deviations in the training sample."
*   Option for vol_stand is used for Goyal and Welch replication --> irrelevant here
*   Pull data from ret_std (target excess returns) and pred_std (volatility standardized predictors) at position idx_start (date position of forecast). In order to assure sufficient observations for different T we go back T observations.
*   Returns two tuples containing train and test samples for X and y for ridge regression.

In [ ]:
def Option_generate_X_y(signals = None, ret_std = None, idx_start = None, T = None, P = None, vol_stand = True):
    # pull T+1 observations --> T observations for train sample and idx_start is the target which we want to forecast
    X = signals[idx_start-T:idx_start+1]
    Y = ret_std[idx_start-T:idx_start+1]

    X_train = X[:-1,:]
    y_train = Y[:-1]

    X_test = X[-1,:].reshape(1,-1)
    y_test = Y[-1]

    if vol_stand:
        std = X_train.std(ddof=0, axis=0)
        X_train = X_train/std
        X_test = X_test/std
        return (X_train,X_test),(y_train,y_test)
    else:
        return (X_train,X_test),(y_train,y_test)

# New Section
This section contains functions that are not used in the rudimentary replicaiton code!

4. pull_data:

*   Pull all results (depending on pull_all) that are relevant for a pre-determined set of parameters. Reads all the files in a local directory and macthes with given parameters to read-in current progress and result.
*   Necessary due to long runtimes of parameter-combinations and risk of program/PC crashing
*   Dependent on path_read which is the directory in which the files are stored
*   Pulls results and coefficient data for $||\hat{\beta}^{2}||$




In [ ]:
def pull_data(params = None, max_iterations = None, path_read = None, pull_all = False):
    ## all possible combinations (ParaGrid)
    combs = list(itertools.product(params["T_grid"], params["P_grid"], params["a_grid"]))
    names_a = ["Results_T{}_P{}_a{}.feather".format(i[0],i[1],i[2]) for i in combs]
    names_b = ["Coef_T{}_P{}_a{}.feather".format(i[0],i[1],i[2]) for i in combs]
    names = names_a, names_b, combs

    ## Dictionary containing results
    results = dict.fromkeys(combs)
    coef = dict.fromkeys(combs)

    # Check which files exist
    for n,m,c in zip(*names):
        e = os.path.isfile(path_read + n)
        f = os.path.isfile(path_read + m)
        # if both exist
        if e and f:
            # if file exists --> load in file
            df_n = pd.read_feather(path_read + n)
            df_n.index = df_n.iloc[:,0]
            df_n = df_n.drop("yyyymm", axis=1)

            df_m = pd.read_feather(path_read + m)
            df_m.index = df_m.iloc[:,0]
            df_m = df_m.drop("yyyymm", axis=1)

            ## if pull all
            if pull_all:
                results[c] = df_n
                coef[c] = df_m
            else:
                ## check if max_iteration has been reached --> if yes, do not write
                if df_n.shape[1] <= max_iterations: results[c] = df_n
                if df_m.shape[1] <= max_iterations: coef[c] = df_m

    return results, coef

5. to_do:
*   Determine progress of results and coefs pulled in previous function
*   Compares the params used as input and the max_iterations in order to determine which parameter-combinations are not yet finished.
*   Output gives the parameter-combinations with the least progress and the number of iterations it has already run.
*   Progress is important so that the correct seed can be applied to future iterations.

=> Will always begin with the parameter-combinations that still need x number of iterations to catch up to other parameter-combinations. After having caught up, run all parameter-combinations together.



In [ ]:
def to_do(results = None, params = None, max_iterations = None):
    # Check what still needs doing --> assume coef and results have same progress (dependent on params) --> some might not exist
    progress = []
    for k,v in results.items():
        if v is None:
            progress.append([0,k])
        elif v.shape[1] < max_iterations:
            progress.append([v.shape[1],k])

    # which combinations need to be run to catch up
    unique = np.unique([i[0] for i in progress])
    unique.sort()
    # if unique is empty list/sequence --> job is done for given params and max_iterations
    if len(unique) == 0:
        return 0
    elif len(unique) == 1:
        iters_needed = (max_iterations - unique)[0]
    else:
        # determine the least number of iterations to catch-up to progress of second-least batch
        # --> necessary in order to uniquely assign seed
        iters_needed = unique[1] - unique[0]

    # minimum progress (for numerically assigning seed_use)
    min_progress = min(unique)
    to_do = []
    for i in progress:
        if i[0] == min_progress: to_do.append(i[1])
    # at which seed to start and iterations needed
    seeds_use = (min_progress,iters_needed)

    return (seeds_use, to_do)

4. run_all:


In [ ]:
def run_all(ret_std = None, pred_std = None, w_i_all = None, params = None, operations = None, end = None):
    from sklearn.linear_model import Ridge
    # combinations to run (taken from operations)
    combs = operations[1]
    # seeds and iterations needed
    seed_start, iters_needed = operations[0]
    # append results to
    dic_res = {i:[] for i in combs}
    dic_coef = {i:[] for i in combs}
    # at what index to stop
    idx_end = np.where(ret_std.index == end)[0][0]
    # define counter
    counter = seed_start
    counter_stop = seed_start + iters_needed

    # Start outer loop (over iterations)
    while counter < counter_stop:
        print("The current counter is: {} out of {} iterations".format(counter,counter_stop))

        # Start second loop over the combinations
        for c in combs:
          # parameters
          T,P,alpha = c
          # generate P-specific signals (used for all P)
          signals = generate_Signals(pred_std=pred_std, P=P, use_seed=counter, w_i_all=w_i_all)
          # lst to append predictions to
          lst_res,lst_coef = [],[]

          # Start third loop over all observations in T
          for t in range(T,idx_end+1): # CHECK IF THIS CORRECT
              # generate XY
              x,y = generate_X_y(signals=signals,ret_std=ret_std, idx_start=t, T=T, P=P)
              # fit and predict
              clf = Ridge(alpha=alpha, fit_intercept=False)
              clf.fit(X=x[0],y=y[0])
              lst_res.append(clf.predict(x[1])[0])
              lst_coef.append(sum(clf.coef_**2))

          ## write to dictionary
          dic_res[c].append(lst_res)
          dic_coef[c].append(lst_coef)
        # update
        counter += 1

    # write new or concatenate dataframes (results)
    for c in combs:
        dic_res[c] = pd.DataFrame(dic_res[c], columns=ret_std.index[c[0]:idx_end+1], index=range(seed_start,counter_stop)).T
        dic_coef[c] = pd.DataFrame(dic_coef[c], columns=ret_std.index[c[0]:idx_end+1], index=range(seed_start,counter_stop)).T

    return dic_res,dic_coef

5. Plot Figure 10

In [ ]:
def plot_fig10(results = None, T = None, P = None, alpha = None, roll_window = None, begin = None, end = None):
    ## Determine start and end of recession periods
    from itertools import groupby
    from operator import itemgetter

    # pull recession data from FRED as dummy variables for a given month
    nber = web.DataReader(["USREC"], "fred", start = begin, end = end)
    _sth = np.where(nber.USREC == 1)[0]
    rec = []
    for k,g in groupby(enumerate(_sth), lambda ix: ix[0] - ix[1]):
        rec.append(list(map(itemgetter(1),g)))
    rec = [(nber.index[i[0]], nber.index[i[-1]]) for i in rec]
    rec_b = [i[0] for i in rec]
    rec_e = [i[1] for i in rec]

    #fig, ax = plt.subplots()
    # relevant combinations
    plt.figure(dpi=2000)
    lbl = []
    cmbs = list(itertools.product(T,[P],[alpha]))
    for n,c in enumerate(cmbs):
        use = results[c].rolling(roll_window).mean().mean(axis=1).dropna()
        use = use.loc[begin:end]
        # plot
        plt.plot(use, lw = 0.65, alpha = 0.8)
        lbl.append("$\hat{\pi},T =$" + str(int(c[0])))
        rec_patches = [plt.axvspan(x1, x2, alpha=0.4, color='grey', zorder=2) for x1, x2 in zip(rec_b, rec_e)]
        #rec_handle = Line2D([0], [0], color='orange', alpha=0.4, lw=4, label='NBER Recession')

    #plt.legend(handles= [rec_handle])
    lbl.append("NBER Recession")
    plt.legend(lbl)
    plt.show()
